In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install catboost
!pip install transformer
from transformers import BertTokenizer, BertModel
# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from transformers import BertTokenizer, BertModel
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Define individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    model = XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    model = AdaBoostClassifier(algorithm='SAMME')
    model.fit(X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    model = SVC()
    model.fit(X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for BERT embedding conversion
text_column = 'answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_data, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'answer': (681, 768)
Running KNN model...
KNN Model Metrics:
Train Accuracy: 0.7668
Train Precision: 0.7554
Train Recall: 0.7668
Train F1 Score: 0.7554
Train Confusion Matrix:
[[ 12  12   8]
 [ 15  65  52]
 [  5  19 288]]

Test Accuracy: 0.6780
Test Precision: 0.6861
Test Recall: 0.6780
Test F1 Score: 0.6812
Test Confusion Matrix:
[[  3   4   6]
 [  2  17  22]
 [  5  27 119]]

Running Random Forest model...
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7415
Test Precision: 0.7237
Test Recall: 0.7415
Test F1 Score: 0.6981
Test Confusion Matrix:
[[  1   4   8]
 [  0  10  31]
 [  0  10 141]]

Running Decision Tree model...
Decision Tree Model Metrics:
Train Accuracy: 0.9937
Tra

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")


# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for BERT embedding conversion
text_column = 'answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_data, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'answer': (681, 768)
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 5}
KNN Model Metrics:
Train Accuracy: 0.7668
Train Precision: 0.7554
Train Recall: 0.7668
Train F1 Score: 0.7554
Train Confusion Matrix:
[[ 12  12   8]
 [ 15  65  52]
 [  5  19 288]]

Test Accuracy: 0.6780
Test Precision: 0.6861
Test Recall: 0.6780
Test F1 Score: 0.6812
Test Confusion Matrix:
[[  3   4   6]
 [  2  17  22]
 [  5  27 119]]

Running Random Forest model...
Best Params: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7268
Test Precision: 0.6323
Test Recall: 0.7268
Test F1 Score: 0.6617
Test Confusion Matrix:
[[ 

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.8235
Train Precision: 0.8299
Train Recall: 0.8235
Train F1 Score: 0.8043
Train Confusion Matrix:
[[  5  18   9]
 [  0  91  41]
 [  0  16 296]]

Test Accuracy: 0.7610
Test Precision: 0.7219
Test Recall: 0.7610
Test F1 Score: 0.7404
Test Confusion Matrix:
[[  0   8   5]
 [  0  23  18]
 [  0  18 133]]

Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7707
Test Precision: 0.7738
Test Recall: 0.7707
Test F1 Score: 0.7453
Test Confusion Matrix:
[[  1   7   5]
 [  0  18  23]
 [  0  12 139]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7332
Train Precision: 0.7534
Train Recall: 0.7332
Train F1 Score: 0.7373
Train Confusion Matrix:
[[ 16  13   3]
 [  1  83  48]
 [  0  62 250]]

Test Accuracy: 0.6293
Test Precision: 0.7047
Test Recall: 0.6293
Test F1 Score: 0.6471
Test Confusion Matrix:
[[  1  10   2]
 [  0  24  17]
 [  2  45 104]]

Running CatBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and text column
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_data, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'cleaned_answer': (681, 768)
Running KNN model...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Params: {'weights': 'uniform', 'n_neighbors': 1}
KNN Model Metrics:
Train Accuracy: 0.9748
Train Precision: 0.9750
Train Recall: 0.9748
Train F1 Score: 0.9749
Train Confusion Matrix:
[[ 31   0   1]
 [  0 128   4]
 [  0   7 305]]

Test Accuracy: 0.6585
Test Precision: 0.6922
Test Recall: 0.6585
Test F1 Score: 0.6709
Test Confusion Matrix:
[[  1   6   6]
 [  2  22  17]
 [  9  30 112]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.7561
Test Precision: 0.6870
Test Recall: 0.7561
Test F1 Score: 0.6906
Test Confusion Matrix:
[[  0   1  12]
 [  0   8  33]
 [  0   4 147]]

Running Decision Tree model...
Best Params: {'min_samples_split': 15, 'min_samples_leaf': 1, 'max_depth': 50}
Decision Tree Model Metrics:
Train Accuracy:

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9771
Train Recall: 0.9769
Train F1 Score: 0.9768
Train Confusion Matrix:
[[ 31   0   1]
 [  0 124   8]
 [  0   2 310]]

Test Accuracy: 0.6488
Test Precision: 0.6646
Test Recall: 0.6488
Test F1 Score: 0.6531
Test Confusion Matrix:
[[  2   5   6]
 [  1  16  24]
 [  3  33 115]]

Running XGBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 1.0, 'n_estimators': 100, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.7268
Test Precision: 0.6484
Test Recall: 0.7268
Test F1 Score: 0.6765
Test Confusion Matrix:
[[  0   3  10]
 [  0   9  32]
 [  1  10 140]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7332
Train Precision: 0.7237
Train Recall: 0.7332
Train F1 Score: 0.7198
Train Confusion Matrix:
[[ 20   6   6]
 [  0  54  78]
 [  0  37 275]]

Test Accuracy: 0.6829
Test Precision: 0.6338
Test Recall: 0.6829
Test F1 Score: 0.6569
Test Confusion Matrix:
[[  0   2  11]
 [  0  13  28]
 [  2  22 127]]

Running CatBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model,
        'CatBoost': catboost_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and text column
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_data, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'cleaned_answer': (681, 768)
Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6244
Test Precision: 0.6406
Test Recall: 0.6244
Test F1 Score: 0.6285
Test Confusion Matrix:
[[  1   4   8]
 [  1  16  24]
 [  3  37 111]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 1}
SVM Model Metrics:
Train Accuracy: 0.9727
Train Precision: 0.9734
Train Recall: 0.9727
Train F1 Score: 0.9724
Train Confusion Matrix:
[[ 31   0   1]
 [  0 121  11]
 [  0   1 311]]

Test Accuracy: 0.6244
Test Precision: 0.6535
Test Recall: 0.6244
Test F1 Score: 0.6370
Test Confusion Matrix:
[[  2   4   7]
 [  3  16  22]
 [  9  32 110]]

Running CatBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca


# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for BERT embedding conversion
text_column = 'answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(bert_data, variance_threshold=0.95)

# Adjust y to ensure labels start from 0
y = original_data['confidence'].apply(lambda x: x + 1)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)


# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'answer': (681, 768)
Number of components selected: 171
Running KNN model...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Params: {'weights': 'uniform', 'n_neighbors': 1}
KNN Model Metrics:
Train Accuracy: 0.9963
Train Precision: 0.9963
Train Recall: 0.9963
Train F1 Score: 0.9963
Train Confusion Matrix:
[[ 32   0   0]
 [  0 145   1]
 [  0   1 365]]

Test Accuracy: 0.6058
Test Precision: 0.6524
Test Recall: 0.6058
Test F1 Score: 0.6191
Test Confusion Matrix:
[[ 1 11  1]
 [ 1 12 14]
 [ 3 24 70]]

Running Random Forest model...
Best Params: {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9963
Train Precision: 0.9963
Train Recall: 0.9963
Train F1 Score: 0.9963
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   0 366]]

Test Accuracy: 0.7153
Test Precision: 0.7021
Test Recall: 0.7153
Test F1 Score: 0.6036
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  1 26]
 [ 0  0 97]]

Running Decision Tree model...
Best Params: {'min_samples_split': 20, 'min_samples_leaf': 1, 'max_depth': 10}
Decision Tree Model Metrics:
Train Accuracy: 0.8915
Train P

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.7923
Train Precision: 0.7882
Train Recall: 0.7923
Train F1 Score: 0.7762
Train Confusion Matrix:
[[  9  13  10]
 [  1  80  65]
 [  0  24 342]]

Test Accuracy: 0.7226
Test Precision: 0.6654
Test Recall: 0.7226
Test F1 Score: 0.6923
Test Confusion Matrix:
[[ 0  8  5]
 [ 0 14 13]
 [ 0 12 85]]

Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 1.0000
Train Precision: 1.0000
Train Recall: 1.0000
Train F1 Score: 1.0000
Train Confusion Matrix:
[[ 32   0   0]
 [  0 146   0]
 [  0   0 366]]

Test Accuracy: 0.7372
Test Precision: 0.6526
Test Recall: 0.7372
Test F1 Score: 0.6896
Test Confusion Matrix:
[[ 0  4  9]
 [ 0 11 16]
 [ 0  7 90]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 150, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.8199
Train Precision: 0.8162
Train Recall: 0.8199
Train F1 Score: 0.8165
Train Confusion Matrix:
[[ 27   4   1]
 [  0  89  57]
 [  0  36 330]]

Test Accuracy: 0.6934
Test Precision: 0.7118
Test Recall: 0.6934
Test F1 Score: 0.6895
Test Confusion Matrix:
[[ 1  8  4]
 [ 2 16  9]
 [ 0 19 78]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.7500
Train Precision: 0.6931
Train Recall: 0.7500
Train F1 Score: 0.7164
Train Confusion Matrix:
[[  0  19  13]
 [  0  70  76]
 [  0  28 338]]

Test Accuracy: 0.7372
Test Precision: 0.6751
Test Recall: 0.7372
Test F1 Score: 0.7044
Test Confusion Matrix:
[[ 0  7  6]
 [ 0 15 12]
 [ 0 11 86]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca


# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for BERT embedding conversion
text_column = 'answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(bert_data, variance_threshold=0.95)

# Adjust y to ensure labels start from 0
y = original_data['confidence'].apply(lambda x: x + 1)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)


# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'answer': (681, 768)
Number of components selected: 171
Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 0.1}
SVM Model Metrics:
Train Accuracy: 0.8548
Train Precision: 0.8531
Train Recall: 0.8548
Train F1 Score: 0.8480
Train Confusion Matrix:
[[ 18   7   7]
 [  1  97  48]
 [  1  15 350]]

Test Accuracy: 0.7518
Test Precision: 0.7594
Test Recall: 0.7518
Test F1 Score: 0.7416
Test Confusion Matrix:
[[ 3  6  4]
 [ 0 15 12]
 [ 1 11 85]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca


# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Specify the text column for BERT embedding conversion
text_column = 'cleaned_answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(bert_data, variance_threshold=0.95)

# Adjust y to ensure labels start from 0
y = original_data['confidence'].apply(lambda x: x + 1)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)


# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'cleaned_answer': (681, 768)
Number of components selected: 168
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 9}
KNN Model Metrics:
Train Accuracy: 0.7206
Train Precision: 0.6909
Train Recall: 0.7206
Train F1 Score: 0.6773
Train Confusion Matrix:
[[  7   8  17]
 [  5  39 102]
 [  1  19 346]]

Test Accuracy: 0.6204
Test Precision: 0.5425
Test Recall: 0.6204
Test F1 Score: 0.5772
Test Confusion Matrix:
[[ 0  2 11]
 [ 1  4 22]
 [ 2 14 81]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9816
Train Precision: 0.9817
Train Recall: 0.9816
Train F1 Score: 0.9816
Train Confusion Matrix:
[[ 30   0   2]
 [  0 141   5]
 [  0   3 363]]

Test Accuracy: 0.7226
Test Precision: 0.6391
Test Recall: 0.7226
Test F1 Score: 

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.7739
Train Precision: 0.7817
Train Recall: 0.7739
Train F1 Score: 0.7363
Train Confusion Matrix:
[[  4  12  16]
 [  0  59  87]
 [  0   8 358]]

Test Accuracy: 0.6715
Test Precision: 0.5713
Test Recall: 0.6715
Test F1 Score: 0.6125
Test Confusion Matrix:
[[ 0  2 11]
 [ 0  5 22]
 [ 0 10 87]]

Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 150, 'learning_rate': 0.01}
XGBoost Model Metrics:
Train Accuracy: 0.9853
Train Precision: 0.9856
Train Recall: 0.9853
Train F1 Score: 0.9852
Train Confusion Matrix:
[[ 31   0   1]
 [  0 139   7]
 [  0   0 366]]

Test Accuracy: 0.6569
Test Precision: 0.5863
Test Recall: 0.6569
Test F1 Score: 0.6085
Test Confusion Matrix:
[[ 1  1 11]
 [ 1  4 22]
 [ 2 10 85]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 150, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7923
Train Precision: 0.7819
Train Recall: 0.7923
Train F1 Score: 0.7792
Train Confusion Matrix:
[[ 30   0   2]
 [  0  67  79]
 [  0  32 334]]

Test Accuracy: 0.6131
Test Precision: 0.5602
Test Recall: 0.6131
Test F1 Score: 0.5855
Test Confusion Matrix:
[[ 0  2 11]
 [ 0  8 19]
 [ 1 20 76]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}
Logistic Regression Model Metrics:
Train Accuracy: 0.7831
Train Precision: 0.7964
Train Recall: 0.7831
Train F1 Score: 0.7526
Train Confusion Matrix:
[[  8   6  18]
 [  0  61  85]
 [  0   9 357]]

Test Accuracy: 0.7299
Test Precision: 0.6422
Test Recall: 0.7299
Test F1 Score: 0.6691
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  8 19]
 [ 0  5 92]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9761
Train Precision: 0.9764
Train Recall: 0.9761
Train F1 Score: 0.9759
Train Confusion Matrix:
[[ 31   0   1]
 [  0 136  10]
 [  0   2 364]]

Test Accuracy: 0.6569
Test Precision: 0.6322
Test Recall: 0.6569
Test F1 Score: 0.6360
Test Confusion Matrix:
[[ 1  5  7]
 [ 0  9 18]
 [ 2 15 80]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from transformers import BertTokenizer, BertModel
import torch
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Mount Google Drive
drive.mount('/content/drive')

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert "ANSWER" column to BERT embeddings and combine with additional features
def convert_to_bert_embeddings(input_file, text_column, feature_columns):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Extract additional features
    additional_features = df[feature_columns].values

    # Combine BERT embeddings with additional features
    combined_features = np.hstack((bert_embeddings, additional_features))

    # Print the shape of the final feature set
    print(f"Combined features shape: {combined_features.shape}")

    # Return the combined features and the original DataFrame
    return pd.DataFrame(combined_features), df
# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }


    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Specify the text column and additional feature columns
text_column = 'answer'
feature_columns = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Convert text column to BERT embeddings and get the combined feature set and original DataFrame
combined_features, original_data = convert_to_bert_embeddings(input_file, text_column, feature_columns)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Best Params: {'subsample': 0.8, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.6981
Test Recall: 0.7366
Test F1 Score: 0.7007
Test Confusion Matrix:
[[  1   2  10]
 [  1  12  28]
 [  0  13 138]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7332
Train Precision: 0.7534
Train Recall: 0.7332
Train F1 Score: 0.7373
Train Confusion Matrix:
[[ 16  13   3]
 [  1  83  48]
 [  0  62 250]]

Test Accuracy: 0.6293
Test Precision: 0.7047
Test Recall: 0.6293
Test F1 Score: 0.6471
Test Confusion Matrix:
[[  1  10   2]
 [  0  24  17]
 [  2  45 104]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}
Logistic Regression Model Metrics:
Train Accuracy: 0.8845
Train Precision: 0.8860
Train Recall: 0.8845
Train F1 Score: 0.8774
Train Confusion Matrix:
[[ 15   7  10]
 [  1 101  30]
 [  0   7 305]]

Test Accuracy: 0.7610
Test Precision: 0.7815
Test Recall: 0.7610
Test F1 Score: 0.7454
Test Confusion Matrix:
[[  1   9   3]
 [  0  21  20]
 [  0  17 134]]

Running SVM model...
Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.6780
Test Precision: 0.7087
Test Recall: 0.6780
Test F1 Score: 0.6910
Test Confusion Matrix:
[[  4   5   4]
 [  2  20  19]
 [ 11  25 115]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from transformers import BertTokenizer, BertModel
import torch
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Mount Google Drive
drive.mount('/content/drive')

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert "ANSWER" column to BERT embeddings and combine with additional features
def convert_to_bert_embeddings(input_file, text_column, feature_columns):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Extract additional features
    additional_features = df[feature_columns].values

    # Combine BERT embeddings with additional features
    combined_features = np.hstack((bert_embeddings, additional_features))

    # Print the shape of the final feature set
    print(f"Combined features shape: {combined_features.shape}")

    # Return the combined features and the original DataFrame
    return pd.DataFrame(combined_features), df
# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }


    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Specify the text column and additional feature columns
text_column = 'cleaned_answer'
feature_columns = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Convert text column to BERT embeddings and get the combined feature set and original DataFrame
combined_features, original_data = convert_to_bert_embeddings(input_file, text_column, feature_columns)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Combined features shape: (681, 771)
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 5}
KNN Model Metrics:
Train Accuracy: 0.7227
Train Precision: 0.7058
Train Recall: 0.7227
Train F1 Score: 0.7082
Train Confusion Matrix:
[[ 11   7  14]
 [  6  60  66]
 [  2  37 273]]

Test Accuracy: 0.6195
Test Precision: 0.5954
Test Recall: 0.6195
Test F1 Score: 0.6072
Test Confusion Matrix:
[[  0   2  11]
 [  2  10  29]
 [  3  31 117]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9916
Train Precision: 0.9917
Train Recall: 0.9916
Train F1 Score: 0.9916
Train Confusion Matrix:
[[ 32   0   0]
 [  0 128   4]
 [  0   0 312]]

Test Accuracy: 0.7415
Test Precision: 0.6640
Test Recall: 0.7415
Test F1 Score: 0.6746
Test Confusion Matrix:
[[  0   0  13]


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9790
Train Precision: 0.9793
Train Recall: 0.9790
Train F1 Score: 0.9789
Train Confusion Matrix:
[[ 31   0   1]
 [  0 124   8]
 [  0   1 311]]

Test Accuracy: 0.6146
Test Precision: 0.6365
Test Recall: 0.6146
Test F1 Score: 0.6236
Test Confusion Matrix:
[[  1   4   8]
 [  3  15  23]
 [  6  35 110]]

Running XGBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 0.8, 'n_estimators': 150, 'learning_rate': 0.01}
XGBoost Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9834
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 31   0   1]
 [  0 126   6]
 [  0   1 311]]

Test Accuracy: 0.7366
Test Precision: 0.6573
Test Recall: 0.7366
Test F1 Score: 0.6757
Test Confusion Matrix:
[[  0   0  13]
 [  0   8  33]
 [  0   8 143]]

Running AdaBoost model...


...

Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7332
Train Precision: 0.7237
Train Recall: 0.7332
Train F1 Score: 0.7198
Train Confusion Matrix:
[[ 20   6   6]
 [  0  54  78]
 [  0  37 275]]

Test Accuracy: 0.6829
Test Precision: 0.6338
Test Recall: 0.6829
Test F1 Score: 0.6569
Test Confusion Matrix:
[[  0   2  11]
 [  0  13  28]
 [  2  22 127]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6390
Test Precision: 0.6461
Test Recall: 0.6390
Test F1 Score: 0.6402
Test Confusion Matrix:
[[  1   4   8]
 [  1  16  24]
 [  4  33 114]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 1}
SVM Model Metrics:
Train Accuracy: 0.9727
Train Precision: 0.9734
Train Recall: 0.9727
Train F1 Score: 0.9724
Train Confusion Matrix:
[[ 31   0   1]
 [  0 121  11]
 [  0   1 311]]

Test Accuracy: 0.6293
Test Precision: 0.6557
Test Recall: 0.6293
Test F1 Score: 0.6413
Test Confusion Matrix:
[[  1   4   8]
 [  4  16  21]
 [ 11  28 112]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from transformers import BertTokenizer, BertModel
import torch
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert "ANSWER" column to BERT embeddings and combine with additional features
def convert_to_bert_embeddings(input_file, text_column, feature_columns):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Extract additional features
    additional_features = df[feature_columns].values

    # Combine BERT embeddings with additional features
    combined_features = np.hstack((bert_embeddings, additional_features))

    # Print the shape of the final feature set
    print(f"Combined features shape: {combined_features.shape}")

    # Return the combined features and the original DataFrame
    return pd.DataFrame(combined_features), df
# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }


    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Specify the text column and additional feature columns
text_column = 'answer'
feature_columns = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Convert text column to BERT embeddings and get the combined feature set and original DataFrame
combined_features, original_data = convert_to_bert_embeddings(input_file, text_column, feature_columns)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(combined_features, variance_threshold=0.95)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Combined features shape: (681, 771)
Number of components selected: 126
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 5}
KNN Model Metrics:
Train Accuracy: 0.7416
Train Precision: 0.7300
Train Recall: 0.7416
Train F1 Score: 0.7340
Train Confusion Matrix:
[[ 11  11  10]
 [ 13  68  51]
 [  5  33 274]]

Test Accuracy: 0.6927
Test Precision: 0.6712
Test Recall: 0.6927
Test F1 Score: 0.6777
Test Confusion Matrix:
[[  3   4   6]
 [  1  11  29]
 [  2  21 128]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7512
Test Precision: 0.6740
Test Recall: 0.7512
Test F1 Score: 0.6922
Test Confusion Matrix:
[[  0   2  11]
 [  0   9  32]
 [  0   6 145]]

Running Decision Tree model...
Best Params: {'min_samples_spli

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.7752
Train Precision: 0.7709
Train Recall: 0.7752
Train F1 Score: 0.7531
Train Confusion Matrix:
[[  6  14  12]
 [  1  71  60]
 [  0  20 292]]

Test Accuracy: 0.7171
Test Precision: 0.6803
Test Recall: 0.7171
Test F1 Score: 0.6981
Test Confusion Matrix:
[[  0  10   3]
 [  1  16  24]
 [  0  20 131]]

Running XGBoost model...
Best Params: {'subsample': 0.8, 'n_estimators': 50, 'learning_rate': 0.01}
XGBoost Model Metrics:
Train Accuracy: 0.9811
Train Precision: 0.9814
Train Recall: 0.9811
Train F1 Score: 0.9808
Train Confusion Matrix:
[[ 28   1   3]
 [  0 127   5]
 [  0   0 312]]

Test Accuracy: 0.7756
Test Precision: 0.7439
Test Recall: 0.7756
Test F1 Score: 0.7468
Test Confusion Matrix:
[[  2   4   7]
 [  1  15  25]
 [  1   8 142]]

Running AdaBoost model...


...

Best Params: {'n_estimators': 150, 'learning_rate': 0.1}
AdaBoost Model Metrics:
Train Accuracy: 0.7164
Train Precision: 0.6468
Train Recall: 0.7164
Train F1 Score: 0.6603
Train Confusion Matrix:
[[  0  17  15]
 [  0  41  91]
 [  0  12 300]]

Test Accuracy: 0.7366
Test Precision: 0.6681
Test Recall: 0.7366
Test F1 Score: 0.6978
Test Confusion Matrix:
[[  0   5   8]
 [  0  13  28]
 [  0  13 138]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.7290
Train Precision: 0.6667
Train Recall: 0.7290
Train F1 Score: 0.6929
Train Confusion Matrix:
[[  0  20  12]
 [  0  62  70]
 [  0  27 285]]

Test Accuracy: 0.7268
Test Precision: 0.6907
Test Recall: 0.7268
Test F1 Score: 0.7079
Test Confusion Matrix:
[[  0   7   6]
 [  0  20  21]
 [  0  22 129]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9874
Train Precision: 0.9875
Train Recall: 0.9874
Train F1 Score: 0.9873
Train Confusion Matrix:
[[ 31   1   0]
 [  0 127   5]
 [  0   0 312]]

Test Accuracy: 0.6927
Test Precision: 0.6754
Test Recall: 0.6927
Test F1 Score: 0.6828
Test Confusion Matrix:
[[  1   7   5]
 [  5  13  23]
 [  0  23 128]]



In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from transformers import BertTokenizer, BertModel
import torch
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert "ANSWER" column to BERT embeddings and combine with additional features
def convert_to_bert_embeddings(input_file, text_column, feature_columns):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Extract additional features
    additional_features = df[feature_columns].values

    # Combine BERT embeddings with additional features
    combined_features = np.hstack((bert_embeddings, additional_features))

    # Print the shape of the final feature set
    print(f"Combined features shape: {combined_features.shape}")

    # Return the combined features and the original DataFrame
    return pd.DataFrame(combined_features), df
# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }


    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Specify the text column and additional feature columns
text_column = 'cleaned_answer'
feature_columns = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Convert text column to BERT embeddings and get the combined feature set and original DataFrame
combined_features, original_data = convert_to_bert_embeddings(input_file, text_column, feature_columns)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(combined_features, variance_threshold=0.95)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Best Params: {'n_estimators': 100, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7752
Train Precision: 0.7819
Train Recall: 0.7752
Train F1 Score: 0.7769
Train Confusion Matrix:
[[ 23   0   9]
 [  0  87  45]
 [  0  53 259]]

Test Accuracy: 0.6146
Test Precision: 0.6211
Test Recall: 0.6146
Test F1 Score: 0.6147
Test Confusion Matrix:
[[  0   4   9]
 [  0  17  24]
 [  2  40 109]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.8151
Train Precision: 0.8100
Train Recall: 0.8151
Train F1 Score: 0.8070
Train Confusion Matrix:
[[ 28   1   3]
 [  1  73  58]
 [  0  25 287]]

Test Accuracy: 0.6878
Test Precision: 0.6771
Test Recall: 0.6878
Test F1 Score: 0.6817
Test Confusion Matrix:
[[  1   5   7]
 [  3  16  22]
 [  3  24 124]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 1}
SVM Model Metrics:
Train Accuracy: 0.8130
Train Precision: 0.8072
Train Recall: 0.8130
Train F1 Score: 0.8057
Train Confusion Matrix:
[[ 28   1   3]
 [  3  75  54]
 [  3  25 284]]

Test Accuracy: 0.6732
Test Precision: 0.6875
Test Recall: 0.6732
Test F1 Score: 0.6798
Test Confusion Matrix:
[[  2   5   6]
 [  4  17  20]
 [  8  24 119]]



In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from transformers import BertTokenizer, BertModel
import torch
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert "ANSWER" column to BERT embeddings and combine with additional features
def convert_to_bert_embeddings(input_file, text_column, feature_columns):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Extract additional features
    additional_features = df[feature_columns].values

    # Combine BERT embeddings with additional features
    combined_features = np.hstack((bert_embeddings, additional_features))

    # Print the shape of the final feature set
    print(f"Combined features shape: {combined_features.shape}")

    # Return the combined features and the original DataFrame
    return pd.DataFrame(combined_features), df
# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }


    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Specify the text column and additional feature columns
text_column = 'answer'
feature_columns = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Convert text column to BERT embeddings and get the combined feature set and original DataFrame
combined_features, original_data = convert_to_bert_embeddings(input_file, text_column, feature_columns)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(combined_features, variance_threshold=0.95)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Best Params: {'n_estimators': 150, 'learning_rate': 0.1}
AdaBoost Model Metrics:
Train Accuracy: 0.7164
Train Precision: 0.6468
Train Recall: 0.7164
Train F1 Score: 0.6603
Train Confusion Matrix:
[[  0  17  15]
 [  0  41  91]
 [  0  12 300]]

Test Accuracy: 0.7366
Test Precision: 0.6681
Test Recall: 0.7366
Test F1 Score: 0.6978
Test Confusion Matrix:
[[  0   5   8]
 [  0  13  28]
 [  0  13 138]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.7311
Train Precision: 0.7354
Train Recall: 0.7311
Train F1 Score: 0.6976
Train Confusion Matrix:
[[  1  19  12]
 [  0  62  70]
 [  0  27 285]]

Test Accuracy: 0.7220
Test Precision: 0.6883
Test Recall: 0.7220
Test F1 Score: 0.7041
Test Confusion Matrix:
[[  0   7   6]
 [  0  20  21]
 [  0  23 128]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 0.1}
SVM Model Metrics:
Train Accuracy: 0.8256
Train Precision: 0.8216
Train Recall: 0.8256
Train F1 Score: 0.8151
Train Confusion Matrix:
[[ 16  10   6]
 [  2  79  51]
 [  1  13 298]]

Test Accuracy: 0.7659
Test Precision: 0.7745
Test Recall: 0.7659
Test F1 Score: 0.7652
Test Confusion Matrix:
[[  4   7   2]
 [  0  23  18]
 [  2  19 130]]

